In [2]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime


In [72]:
##################################################
# Load transaction 
##################################################

transactions = utils.read_multiple_csv('../input/preprocessed_data/transactions')


100%|██████████| 4/4 [00:27<00:00,  6.85s/it]


In [1]:
transactions = transactions.head(n = 1000)

NameError: name 'transactions' is not defined

In [24]:
s = time.time()
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
e = time.time()
e-s # 469.64

469.64119505882263

In [73]:
##################################################
# Load train set user and test set user
##################################################
train = pd.read_csv('../input/train.csv') # 這個檔案不用, 因為不知道他的label日期怎麼取
train_v2 = pd.read_csv('../input/train_v2.csv') #只用這個檔案, 然後做data augmentation
test = pd.read_csv('../input/sample_submission_v2.csv')


In [7]:
############################################
# 找出那些ms在train和train_v2是不一致的
############################################
for msno, is_churn in train_v2[train_v2.msno.isin(train.msno)].values[:10,:]:
    label_from_v1 = train[train.msno == msno]['is_churn'].iloc[0]
    label_from_v2 = train_v2[train_v2.msno == msno]['is_churn'].iloc[0]
    #print (label_from_v1 == label_from_v2)
    if (label_from_v1 == label_from_v2) == False:
        print (msno)
    #print (label_from_v1.iloc[0])

K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=
ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=
kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=
dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=
otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=
dfLS2/Pom6O3iUpo+mf/K1Bc6g+7LFKzoNf+zlJtDoQ=
a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=


In [12]:
train[train.msno == 'K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=']

,msno,is_churn
758038,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0


In [13]:
train_v2[train_v2.msno == 'K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=']

,msno,is_churn
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [5]:
##################################################
# 善用time(時間差)來區隔交易日期時間點的行為
##################################################

def t_diff(x, time_diff):
    # x : the dataframe belong to a each specific user
    # time_diff: 2,3,4,...
    for i in range(1, time_diff):
        x['t-{}_transaction_date'.format(i)] = x['transaction_date'].shift(i)
        x['t-{}_membership_expire_date'.format(i)] = x['membership_expire_date'].shift(i)
        x['t-{}_payment_method_id'.format(i)] = x['payment_method_id'].shift(i)
        x['t-{}_payment_plan_days'.format(i)] = x['payment_plan_days'].shift(i)
        x['t-{}_plan_list_price'.format(i)] = x['plan_list_price'].shift(i)
        x['t-{}_actual_amount_paid'.format(i)] = x['actual_amount_paid'].shift(i)
        x['t-{}_is_auto_renew'.format(i)] = x['is_auto_renew'].shift(i)
        x['t-{}_is_cancel'.format(i)] = x['is_cancel'].shift(i)
    return x
def make_order_number(x):
    x['order_number'] = [i+1 for i in range(x.shape[0])]
    return x

def days_since_the_last_expiration(x):
    # x: each row of dataframe, series
    # return 負的代表消費者提早續約在上次交易到期之前,正的代表他在上次到期後才續約,0代表剛好那天續約
    if x['t-1_membership_expire_date']== np.nan:
        return np.nan
    else:
        return (x.transaction_date - x['t-1_membership_expire_date']).days
def days_since_the_last_subscription(x):
    # x: each row of dataframe, series
    # return 
    if x['t-1_transaction_date']== np.nan:
        return np.nan
    else:
        return (x.transaction_date - x['t-1_transaction_date']).days
def is_subscribe_early(x):
    # x: each row of dataframe, series
    if x['order_number']== 1:
        return np.nan
    else: 
        if x.days_since_the_last_expiration < 0:
            return 1
        else:
            return 0
def helper_function(x):
    #
    x['days_since_the_last_expiration'] = x.apply(days_since_the_last_expiration,axis = 1)
    x['days_since_the_last_subscription'] = x.apply(days_since_the_last_subscription,axis = 1)
    x['is_subscribe_early'] = x.apply(is_subscribe_early,axis = 1)
    return x



# user = ''K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA='

In [74]:
user = transactions[transactions.msno == 'K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)

col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
2207543,2015-01-24,2015-07-29,0,1
2207544,2015-02-24,2015-08-26,0,2
2207545,2015-03-24,2015-09-26,0,3
2207546,2015-04-24,2015-10-26,0,4
2207547,2015-04-28,2015-10-29,0,5
2207548,2015-04-28,2015-09-29,1,6
2207549,2015-05-27,2015-11-29,0,7
2207550,2015-06-27,2015-12-29,0,8
2207551,2015-07-27,2016-01-29,0,9
2207552,2015-08-27,2016-02-29,0,10


# check point

In [9]:

from datetime import datetime, timedelta

datetime.strptime('2017-02-3', '%Y-%m-%d') + timedelta(30)

datetime.datetime(2017, 3, 5, 0, 0)

# user = ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=

In [75]:
user = transactions[transactions.msno == 'ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
5247353,2015-04-05,2015-05-04,0,1
5247354,2015-05-05,2015-06-04,0,2
5247355,2015-06-06,2015-06-04,1,3
5247356,2015-10-15,2015-11-14,0,4
5247357,2015-11-15,2015-12-14,0,5
5247358,2015-12-15,2016-01-14,0,6
5247359,2016-01-15,2016-02-14,0,7
5247360,2016-02-15,2016-03-14,0,8
5247361,2016-03-15,2016-04-14,0,9
5247362,2016-04-15,2016-05-14,0,10


# check point

In [13]:
from datetime import datetime, timedelta

datetime.strptime('2017-02-14', '%Y-%m-%d') + timedelta(30)

datetime.datetime(2017, 3, 16, 0, 0)

In [12]:
train[train.msno == 'ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=']

,msno,is_churn
758043,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0


In [13]:
train_v2[train_v2.msno == 'ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=']

,msno,is_churn
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1


In [23]:
from datetime import datetime, timedelta

datetime.strptime('2017-02-14', '%Y-%m-%d') + timedelta(30)

datetime.datetime(2017, 3, 16, 0, 0)

# user =  kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=

In [76]:
user = transactions[transactions.msno == 'kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)

col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
197916,2016-07-20,2016-08-20,0,1
197917,2016-08-20,2016-09-20,0,2
197918,2016-09-20,2016-10-20,0,3
197919,2016-10-20,2016-11-20,0,4
197920,2016-11-20,2016-12-20,0,5
197921,2016-12-20,2017-01-20,0,6
197922,2017-01-20,2017-02-20,0,7
197923,2017-02-20,2017-03-20,0,8
197924,2017-03-07,2017-03-07,1,9


In [15]:
datetime.strptime('2017-02-20', '%Y-%m-%d') + timedelta(30)

datetime.datetime(2017, 3, 22, 0, 0)

In [25]:
train[train.msno == 'kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=']

,msno,is_churn
758047,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,0


In [26]:
train_v2[train_v2.msno == 'kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=']

,msno,is_churn
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1


In [16]:
# rules generating training label

# 1.在2月份到期的那個時間點(current membership expires)之後的30天稱作檢查點
# 2.在三月份發生交易的時間點(future)超過check points--->churn
# 3.在三月份發生交易的時間點(future)到此交易過期的時間點的這段日子即使有在check points內,但消費者作出cancel的決定也是--->churn

# user =  dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=


In [77]:
user = transactions[transactions.msno == 'dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)

col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
3409726,2016-02-03,2016-03-04,0,1
3409727,2016-03-02,2016-04-03,0,2
3409728,2016-04-02,2016-07-12,0,3
3409729,2016-07-06,2016-10-10,0,4
3409730,2016-07-06,2016-09-10,0,5
3409731,2016-07-06,2016-08-11,0,6
3409732,2016-10-05,2017-02-07,0,7
3409733,2017-01-09,2017-03-09,0,8
3409734,2017-03-07,2017-04-08,0,9
3409735,2017-03-28,2017-07-07,0,10


In [18]:
datetime.strptime('2017-02-07', '%Y-%m-%d') + timedelta(30)
# 這個人在三月份發生兩次交易,雖然都沒有cancel,但他第二次發生交易的時間點超過check points
# 這個人在三月份發生兩次交易以上,我們以三月份交易的最後一次來當當作判斷他有沒有超過check points

datetime.datetime(2017, 3, 9, 0, 0)

In [19]:
train_v2[train_v2.msno == 'dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=']

,msno,is_churn
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1


# user = otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=

In [78]:
user = transactions[transactions.msno == 'otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)

col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
1759792,2015-01-13,2015-02-12,0,1
1759793,2015-02-23,2015-03-25,0,2
1759794,2015-03-30,2015-04-29,0,3
1759795,2015-04-30,2015-05-30,0,4
1759796,2015-06-03,2015-07-03,0,5
1759797,2015-07-04,2015-08-03,0,6
1759798,2015-08-05,2015-09-04,0,7
1759799,2015-09-13,2015-10-13,0,8
1759800,2015-10-14,2015-11-13,0,9
1759801,2015-11-14,2015-12-14,0,10


In [79]:
datetime.strptime('2017-02-24', '%Y-%m-%d') + timedelta(30) # 這個人落在cp之內但也是流失還是等號也算流失？

datetime.datetime(2017, 3, 26, 0, 0)

In [81]:
train_v2[train_v2.msno == 'otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=']

,msno,is_churn
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1


# user = '4z+AckeOwZ2Y5QNzRx4QlXaEqvCmq9HAaf1G0eDB2R0='

In [83]:
train_v2[train_v2.msno == '4z+AckeOwZ2Y5QNzRx4QlXaEqvCmq9HAaf1G0eDB2R0=']

,msno,is_churn
7961,4z+AckeOwZ2Y5QNzRx4QlXaEqvCmq9HAaf1G0eDB2R0=,1


In [88]:
user = transactions[transactions.msno == '4z+AckeOwZ2Y5QNzRx4QlXaEqvCmq9HAaf1G0eDB2R0=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
# user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
# user = user.groupby('msno').apply(helper_function)

col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]
#這個人到到期日沒有落在2月份的怎麼會在train set裡面, 無法判斷...noise

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
2510726,2017-03-12,2017-03-22,0,1
2510727,2017-03-16,2017-08-09,0,2
2510728,2017-03-16,2017-05-10,0,3
2510729,2017-03-16,2017-05-31,0,4
2510730,2017-03-16,2017-04-19,0,5
2510731,2017-03-16,2017-06-21,0,6
2510732,2017-03-16,2017-03-29,0,7
2510733,2017-03-16,2017-07-19,0,8
2510734,2017-03-16,2017-05-03,0,9
2510735,2017-03-16,2017-06-28,0,10


# user = 791c32ArWs05ZCpF+brfdTcgsMsOIKIxMCUe1Qh4OKE=

In [96]:
s = time.time()
user = transactions[transactions.msno == '791c32ArWs05ZCpF+brfdTcgsMsOIKIxMCUe1Qh4OKE=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
e = time.time()
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,transaction_date,membership_expire_date,is_cancel,order_number
3295675,2015-04-17,2015-05-17,0,1
3295676,2015-05-18,2015-06-17,0,2
3295677,2015-06-18,2015-07-18,0,3
3295678,2015-08-29,2015-09-28,0,4
3295679,2015-09-30,2015-10-30,0,5
3295680,2015-11-02,2015-12-02,0,6
3295681,2015-12-03,2016-01-02,0,7
3295682,2016-02-21,2016-02-28,0,8
3295683,2016-02-24,2016-03-06,0,9
3295684,2016-02-24,2016-03-13,0,10


In [70]:
datetime.strptime('2017-02-07', '%Y-%m-%d') + timedelta(30)

datetime.datetime(2017, 3, 9, 0, 0)

# user = a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=

In [97]:
s = time.time()
user = transactions[transactions.msno == 'a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
e = time.time()
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,transaction_date,membership_expire_date,is_cancel,order_number
2198325,2015-01-24,2015-02-24,0,1
2198326,2015-02-24,2015-03-24,0,2
2198327,2015-03-24,2015-04-24,0,3
2198328,2015-04-24,2015-05-24,0,4
2198329,2015-05-24,2015-06-24,0,5
2198330,2015-06-24,2015-07-24,0,6
2198331,2015-07-24,2015-08-24,0,7
2198332,2015-08-24,2015-09-24,0,8
2198333,2015-09-24,2015-10-24,0,9
2198334,2015-10-24,2015-11-24,0,10


In [99]:
datetime.strptime('2017-02-24', '%Y-%m-%d')+ timedelta(30)

datetime.datetime(2017, 3, 26, 0, 0)

# user = qP66pCFEiIONygGX9baEJQ5JKpVyFo2ao0AmTuw5ACY=


# 以下沒在官方文件發現, 是自己推斷（切記過分推斷, 怪異的都做noise不考慮)

    # 1.如果可以當作預測的order發生兩次, 以最後一筆order當作判斷label的依據

    # 2.如果剛好發生在cp(等號)貌似是churn

In [101]:
s = time.time()
user = transactions[transactions.msno == 'qP66pCFEiIONygGX9baEJQ5JKpVyFo2ao0AmTuw5ACY=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
e = time.time()
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,transaction_date,membership_expire_date,is_cancel,order_number
2305349,2016-02-06,2017-03-22,0,1
2305350,2017-03-20,2017-04-21,0,2
2305351,2017-03-20,2017-11-17,0,3
2305352,2017-03-20,2017-08-19,0,4
2305353,2017-03-20,2018-01-16,0,5
2305354,2017-03-20,2017-10-18,0,6
2305355,2017-03-20,2018-02-15,0,7
2305356,2017-03-20,2017-09-18,0,8
2305357,2017-03-20,2017-06-20,0,9
2305358,2017-03-20,2017-07-20,0,10


In [104]:
datetime.strptime('2017-02-24', '%Y-%m-%d')+ timedelta(30)

datetime.datetime(2017, 3, 26, 0, 0)

In [105]:
train_v2[train_v2.msno == 'qP66pCFEiIONygGX9baEJQ5JKpVyFo2ao0AmTuw5ACY='] # noise

,msno,is_churn
46936,qP66pCFEiIONygGX9baEJQ5JKpVyFo2ao0AmTuw5ACY=,1


# user = WBpgK5pHlEe6kkc73mkEIXRsPUSFuuIImZ8tD9rQ7C0=


In [106]:
train_v2[train_v2.msno == 'WBpgK5pHlEe6kkc73mkEIXRsPUSFuuIImZ8tD9rQ7C0=']

,msno,is_churn
22951,WBpgK5pHlEe6kkc73mkEIXRsPUSFuuIImZ8tD9rQ7C0=,1


In [102]:
user = transactions[transactions.msno == 'WBpgK5pHlEe6kkc73mkEIXRsPUSFuuIImZ8tD9rQ7C0=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
e = time.time()
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
794184,2015-10-26,2015-12-01,0,1
794185,2015-11-26,2015-12-31,0,2
794186,2015-12-26,2016-01-31,0,3
794187,2016-01-26,2016-03-02,0,4
794188,2016-02-26,2016-03-31,0,5
794189,2016-03-26,2016-05-01,0,6
794190,2016-04-26,2016-05-31,0,7
794191,2016-05-26,2016-07-01,0,8
794192,2016-06-26,2016-07-31,0,9
794193,2016-07-26,2016-08-31,0,10


In [103]:
datetime.strptime('2017-02-28', '%Y-%m-%d')+ timedelta(30)

datetime.datetime(2017, 3, 30, 0, 0)

# user = MFrEH81DqeRbBIYFssxgeyQ0J5eq0JCQSt6jkx+9/ps=

In [107]:
train_v2[train_v2.msno == 'MFrEH81DqeRbBIYFssxgeyQ0J5eq0JCQSt6jkx+9/ps=']

,msno,is_churn
608515,MFrEH81DqeRbBIYFssxgeyQ0J5eq0JCQSt6jkx+9/ps=,1


In [109]:
user = transactions[transactions.msno == 'MFrEH81DqeRbBIYFssxgeyQ0J5eq0JCQSt6jkx+9/ps=']
##################################################
# Convert string to datetime format
##################################################
user['membership_expire_date']  = user.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
user['transaction_date']  = user.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# For membership_loyalty
##################################################
user['how_long_it_will_expire'] = [i.days for i in (user.membership_expire_date - user.transaction_date)]
user = user.groupby('msno').apply(t_diff, 2)
user = user.groupby('msno').apply(make_order_number)
user = user.groupby('msno').apply(helper_function)
e = time.time()
col = ['transaction_date','membership_expire_date','is_cancel','order_number']
user[col]


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vi

,transaction_date,membership_expire_date,is_cancel,order_number
2975093,2017-02-28,2017-03-10,0,1
2975094,2017-03-11,2017-04-29,0,2
2975095,2017-03-11,2017-07-01,0,3
2975096,2017-03-11,2017-05-06,0,4
2975097,2017-03-11,2017-04-15,0,5
2975098,2017-03-11,2017-05-13,0,6
2975099,2017-03-11,2017-03-25,0,7
2975100,2017-03-11,2017-06-03,0,8
2975101,2017-03-11,2017-06-10,0,9
2975102,2017-03-11,2017-04-08,0,10


In [ ]:
#把到期日沒有在二月的人都filter

In [ ]:
def f_():
    df = [(transactions.membership_expire_date >= datetime.strptime('2017-02-01', '%Y-%m-%d')  )
                 &(transactions.membership_expire_date <= datetime.strptime('2017-02-28', '%Y-%m-%d')  )
                 ] # 找出這個人在二月份中的交易紀錄
    if df.empty:
        
transactions.groupby('msno').apply(f_)

In [ ]:
df[df.column.isin(list-object)]

In [110]:
datetime.strptime('2017-02-01', '%Y-%m-%d') 

datetime.datetime(2017, 2, 1, 0, 0)

In [113]:
datetime.strptime('2017-02-28', '%Y-%m-%d') 

datetime.datetime(2017, 2, 28, 0, 0)

In [ ]:
transactions.drop(transactions[transactions.membership_expire_date < ])

In [ ]:
df = df.drop(df[df.score < 50].index)



In [100]:
# 找出三月份(future)發生兩筆交易的人且也在train_v2中的
#
d = transactions[(transactions.transaction_date >= datetime.strptime('2017-02-01', '%Y-%m-%d') )
                 &(transactions.transaction_date <=datetime.strptime('2017-04-30', '%Y-%m-%d') )
                 ] #1.找出在三月份交易的人
d = d[d.msno.isin(train_v2.msno)]

TypeError: '>=' not supported between instances of 'str' and 'datetime.datetime'